In [552]:
import os
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import humanize

In [553]:
data_dir = './data_dump'
pd.set_option('display.max_columns', None)

In [554]:
def make_hover_data(dataframe):
    # Список обязательных колонокdef make_hover_data(dataframe):
  # Список обязательных колонок
  base_cols = [
    'timing.frame',
    'projectile.flight_time',
    'projectile.velocity.z',
    'projectile.velocity.y',
    'projectile.velocity.x',
    'projectile.velocity.length',
    'projectile.mach_number',
    'projectile.current_drag_coef',
    'projectile.kinetic_energy',  # будет обработан отдельно
    'projectile.position.z',
    'projectile.position.y',
    'projectile.position.x'
  ]
  
  # Создаём копию, добавляем отсутствующие как NaN
  safe_dataframe = dataframe.copy()
  for col in base_cols:
    if col not in safe_dataframe.columns:
      safe_dataframe[col] = -1

  # Человеко-читаемая энергия
  energy_col = safe_dataframe['projectile.kinetic_energy'].apply(
    lambda x: f"{humanize.metric(x)}J" if pd.notnull(x) else "–"
  )

  # Сборка customdata
  customdata = pd.concat([
    safe_dataframe[[
      'timing.frame',
      'projectile.flight_time',
      'projectile.velocity.z',
      'projectile.velocity.y',
      'projectile.velocity.x',
      'projectile.velocity.length',
      'projectile.mach_number',
      'projectile.current_drag_coef'
    ]],
    energy_col,
    safe_dataframe[[
      'projectile.position.z',
      'projectile.position.y',
      'projectile.position.x'
    ]].abs()
  ], axis=1)

  # Готовый шаблон
  hovertemplate = (
    "Frame %{customdata[0]} (%{customdata[1]:.5f}s)<br>" +
    "Dist: %{customdata[9]:.2f} m | Drop: %{customdata[10]:.3f} m | Drift: %{customdata[11]:.5f} m<br>" +
    "Rate: (%{customdata[2]:.4f}, %{customdata[3]:.4f}, %{customdata[4]:.4f}) m/s<br>" +
    "Velocity: %{customdata[5]:.2f} m/s (%{customdata[6]:.3f} M)<br>" +
    "Kinetic Energy: %{customdata[8]}<br>" +
    "Cd: %{customdata[7]:.3f}" +
    "<extra></extra>"
  )

  return customdata, hovertemplate

In [555]:
def match_by_nearest(sample_times, df_times, df, tolerance=None):
  matched_rows = []
  used_indices = set()
  
  for t in sample_times:
    # Вычислить разницу
    differences = np.abs(df_times - t)
    
    # Исключить уже использованные индексы
    differences[list(used_indices)] = np.inf
    
    # Найти ближайший индекс
    best_idx = differences.idxmin()
    
    # Проверка на допуск (если нужно)
    if tolerance is not None and differences[best_idx] > tolerance:
      matched_rows.append(None)
    else:
      matched_rows.append(df.loc[best_idx])
      used_indices.add(best_idx)
  
  # Вернуть как DataFrame, пропуская None
  return pd.DataFrame(matched_rows).dropna()

In [556]:
def prepare_sample(dataframe):
  dataframe['projectile.position.x'] = -dataframe['windage_in'] / 39.37
  dataframe['projectile.position.y'] = dataframe['elev_in'] / 39.37
  dataframe['projectile.position.z'] = -dataframe['range_yd'] * 0.9144
  dataframe['projectile.kinetic_energy'] = dataframe['energy_ft_lbf'] * 1.3558
  dataframe['projectile.velocity.length'] = dataframe['vel_xy_ft_s'] * 0.3048
  dataframe['projectile.mach_number'] = dataframe['projectile.velocity.length'] / (1116.0 * 0.3048)
  return dataframe

In [557]:
sample_m855 = pd.read_csv('./sample_m855_62gr.csv')
sample_9mm = pd.read_csv('./samlpe_9x19_plus_p.csv')
sample_m855 = prepare_sample(sample_m855)
sample_9mm = prepare_sample(sample_9mm)

In [558]:
files = os.listdir(data_dir)
#   with open(os.path.join(data_dir, fn), 'r') as f:
#     data = json.load(f)
#     df_raw = pd.json_normalize(data)
with open(os.path.join(data_dir, files[-1]), 'r') as f:
  data = json.load(f)
  df_raw = pd.json_normalize(data)

In [559]:
static_data = {}
static_cols = ['angular_velocity', 'position', 'velocity', 'rotation']
for col in df_raw.columns:
  if len(df_raw[col].unique()) == 1 and all(stc not in col for stc in static_cols):
    static_data[col] = df_raw[col].unique()[0]
df = df_raw.drop(list(static_data.keys()), axis=1)
static_data

{'medium.base_density': np.float64(1050.0),
 'medium.damping': np.float64(0.5),
 'medium.poisson_ratio': np.float64(0.49),
 'medium.rha_coef': np.float64(0.04),
 'medium.type': 'solid',
 'medium.young_modulus': np.float64(100000.0),
 'naming.ammo_name': '5.56x45_M855',
 'naming.projectile_uid': 'proj_BqhS9inx',
 'naming.weapon_name': 'SCAR_L_CQC',
 'projectile.ammo': '<Resource#-9223371995398273689>',
 'projectile.caliber': np.float64(5.7),
 'projectile.core_caliber': np.float64(5.4),
 'projectile.core_hardness': np.float64(0.6),
 'projectile.core_mass': np.float64(2.0),
 'projectile.cross_section': np.float64(2.55175863287831e-05),
 'projectile.drag_coef': np.float64(0.28),
 'projectile.fragmentation_chance': np.float64(0.1),
 'projectile.fragmentation_count': np.int64(0),
 'projectile.fragments_max': np.int64(4),
 'projectile.fragments_min': np.int64(2),
 'projectile.impact_count': np.int64(0),
 'projectile.kind': 'normal',
 'projectile.length': np.float64(23.0),
 'projectile.length_

In [ ]:
df['projectile.velocity.length'] = (df['projectile.velocity.x']**2 + df['projectile.velocity.y']**2 + df['projectile.velocity.z']**2)**0.5
df['projectile.position.length'] = (df['projectile.position.x']**2 + df['projectile.position.y']**2 + df['projectile.position.z']**2)**0.5
df['projectile.kinetic_energy'] = 0.5 * static_data['projectile.mass'] * df['projectile.velocity.length'] ** 2 *0.001
df['projectile.mach_number'] = df['projectile.velocity.length'] / static_data['medium.speed_of_sound']
df.columns

KeyError: 'medium.speed_of_sound'

In [ ]:
fig = go.Figure()

marker_step = 10
marker_indices = df.index[::marker_step]
mach_min = 0.0
mach_max = 3.0
mach_threshold = 1.0
mach_colorscale = [
    [mach_min, '#f29'],   # Зеленый при Mach = 0.0
    [mach_threshold / mach_max, '#4af'],  # Голубой при Mach = 1.0
    [mach_threshold / mach_max, '#f00'],  # Красный тоже при Mach = 1.0 (резкое изменение)
    [1.0, '#ff0']    # Желтый при Mach = 3.0
]

# fig.add_trace(go.Scatter3d(
#   x=df.loc[marker_indices, 'projectile.position.x'],
#   z=df.loc[marker_indices, 'projectile.position.y'],
#   y=df.loc[marker_indices, 'projectile.position.z'],
#   mode='markers',
#   marker=dict(
#     size=3,
#     color=df['projectile.current_drag_coef'][2:],
#     colorscale='Jet',
#     colorbar=dict(
#       title='Drag Coefficient',
#       y=0.0,
#       len=0.5
#     )
#   ),
#   name="Drag Coefficient"
# ))
customdata_df, hovertemplate_df = make_hover_data(df)
fig.add_trace(go.Scatter3d(
  x=df['projectile.position.x'],
  z=df['projectile.position.y'],
  y=df['projectile.position.z'],
  mode='lines',
  line=dict(
    width=5,
    color=df['projectile.mach_number'],
    colorscale=mach_colorscale,
    cmin=mach_min,
    cmax=mach_max,
    colorbar=dict(
      title='Mach Number',
      y=0.5,
      len=0.5
    ),
  ),
  name='YIP',
  customdata=customdata_df,
  hovertemplate=hovertemplate_df
))

###
# Draw Sample Plot
###

# sample = sample_m855
# customdata_sample, hovertemplate_sample = make_hover_data(sample)
# fig.add_trace(go.Scatter3d(
#   x=sample['projectile.position.x']+0.01,
#   z=sample['projectile.position.y'],
#   y=sample['projectile.position.z'],
#   mode='lines',
#   line=dict(
#       width=5,
#       cmin=mach_min,
#       cmax=mach_max,
#       color=sample['projectile.mach_number'],
#       colorscale=mach_colorscale,
#     ),
#   name='Sample',
#   customdata=customdata_sample,
#   hovertemplate=hovertemplate_sample
# ))


scene_ax = dict(
  backgroundcolor='#444',
  gridcolor='#666',
  showbackground=True,
  zerolinecolor='#666',
  color='white'
)
fig.update_layout(
  paper_bgcolor='#333',
  plot_bgcolor='#444',
  font=dict(
    color='white'
  ),
  scene_camera=dict(
    eye=dict(x=-1.5, y=0.0, z=0.0)  # положение "глаза камеры"
  ),
  scene=dict(
    aspectmode='manual',
    aspectratio=dict(x=0.5, y=2.5, z=0.5),
    xaxis=scene_ax,
    yaxis=scene_ax,
    zaxis=scene_ax
  ),
  margin=dict(l=0, r=0, b=0, t=10)
)

In [ ]:
cols = ['projectile.flight_time', 'projectile.position.z', 'projectile.kinetic_energy', 'projectile.velocity.length', 'projectile.mach_number']
sample_times = sample['projectile.flight_time'].values
df_times = df['projectile.flight_time']

matched_df = match_by_nearest(sample_times, df_times, df)[cols].reset_index(drop=True)
matched_df - sample[cols]

,projectile.flight_time,projectile.position.z,projectile.kinetic_energy,projectile.velocity.length,projectile.mach_number
0,0.000000e+00,0.000000,157.267600,45.700000,-1.954033
1,-9.714451e-17,43.598116,-1235.091813,-779.668536,-2.302464
2,-5.666667e-02,89.303283,-978.848755,-693.584919,-2.048998
3,-1.283333e-01,135.030628,-765.986630,-613.032044,-1.812381
4,-2.116667e-01,180.765752,-592.440896,-538.174185,-1.592724
5,-3.033333e-01,226.493543,-452.791671,-470.411958,-1.393734
6,-4.050000e-01,272.221495,-347.037327,-410.570408,-1.218039
7,-5.266667e-01,317.949616,-269.754656,-361.697263,-1.074599
8,-6.583333e-01,363.677911,-219.587845,-325.925825,-0.969686
9,-8.000000e-01,409.406389,-188.399797,-300.403908,-0.895162
